## Project 🚧

The marketing team needs help on a new project. After doing some user research, the team discovered that **70% of their users who are planning a trip would like to have more information about the destination they are going to**. 

In addition, user research shows that **people tend to be defiant about the information they are reading if they don't know the brand** which produced the content. 

Therefore, Kayak Marketing Team would like to create an application that will recommend where people should plan their next holidays. The application should be based on real data about:

* Weather 
* Hotels in the area 

The application should then be able to recommend the best destinations and hotels based on the above variables at any given time. 

## Goals 🎯

As the project has just started, your team doesn't have any data that can be used to create this application. Therefore, your job will be to: 

* Scrape data from destinations 
* Get weather data from each destination 
* Get hotels' info about each destination
* Store all the information above in a data lake
* Extract, transform and load cleaned data from your datalake to a data warehouse

## Scope of this project 🖼️

Marketing team wants to focus first on the best cities to travel to in France. According <a href="https://one-week-in.com/35-cities-to-visit-in-france/" target="_blank">One Week In.com</a> here are the top-35 cities to visit in France: 

```python 
["Mont Saint Michel",
"St Malo",
"Bayeux",
"Le Havre",
"Rouen",
"Paris",
"Amiens",
"Lille",
"Strasbourg",
"Chateau du Haut Koenigsbourg",
"Colmar",
"Eguisheim",
"Besancon",
"Dijon",
"Annecy",
"Grenoble",
"Lyon",
"Gorges du Verdon",
"Bormes les Mimosas",
"Cassis",
"Marseille",
"Aix en Provence",
"Avignon",
"Uzes",
"Nimes",
"Aigues Mortes",
"Saintes Maries de la mer",
"Collioure",
"Carcassonne",
"Ariege",
"Toulouse",
"Montauban",
"Biarritz",
"Bayonne",
"La Rochelle"]
```

Your team should focus **only on the above cities for your project**. 


## Helpers 🦮

To help you achieve this project, here are a few tips that should help you

### Get weather data with an API 

*   Use https://nominatim.org/ to get the gps coordinates of all the cities (no subscription required) Documentation : https://nominatim.org/release-docs/develop/api/Search/

*   Use https://openweathermap.org/appid (you have to subscribe to get a free apikey) and https://openweathermap.org/api/one-call-api to get some information about the weather for the 35 cities and put it in a DataFrame

*   Determine the list of cities where the weather will be the nicest within the next 7 days For example, you can use the values of daily.pop and daily.rain to compute the expected volume of rain within the next 7 days... But it's only an example, actually you can have different opinions on a what a nice weather would be like 😎 Maybe the most important criterion for you is the temperature or humidity, so feel free to change the rules !

*   Save all the results in a `.csv` file, you will use it later 😉 You can save all the informations that seem important to you ! Don't forget to save the name of the cities, and also to create a column containing a unique identifier (id) of each city (this is important for what's next in the project)

*   Use plotly to display the best destinations on a map

### Scrape Booking.com 

Since BookingHoldings doesn't have aggregated databases, it will be much faster to scrape data directly from booking.com 

You can scrap as many information asyou want, but we suggest that you get at least:

*   hotel name,
*   Url to its booking.com page,
*   Its coordinates: latitude and longitude
*   Score given by the website users
*   Text description of the hotel


### Create your data lake using S3 

Once you managed to build your dataset, you should store into S3 as a csv file. 

### ETL 

Once you uploaded your data onto S3, it will be better for the next data analysis team to extract clean data directly from a Data Warehouse. Therefore, create a SQL Database using AWS RDS, extract your data from S3 and store it in your newly created DB.

## Deliverable 📬

To complete this project, your team should deliver:

* A `.csv` file in an S3 bucket containing enriched information about weather and hotels for each french city

* A SQL Database where we should be able to get the same cleaned data from S3 

* Two maps where you should have a Top-5 destinations and a Top-20 hotels in the area. You can use plotly or any other library to do so. It should look something like this: 

![Map](https://full-stack-assets.s3.eu-west-3.amazonaws.com/images/Kayak_best_destination_project.png)

In [1]:
!pip install beautifulsoup4
from bs4 import BeautifulSoup #on importe la classe Beautifulsoup depuis la librairie beautifulsoup
import pandas as pd
import json
from datetime import datetime
import requests #permet de faire des requêtes http
#soup = BeautifulSoup("<html>data</html>", "html.parser") # Here we used an HTML Parser

In [2]:
#r = requests.get("https://nominatim.openstreetmap.org/?city=paris&country=France&format=json") #q=quary
#r

In [3]:
#list of cities to be studied
list_cities = ["Mont Saint Michel", "St Malo","Bayeux", "Le Havre", "Rouen", "Paris", "Amiens", "Lille",
               "Strasbourg", "Chateau du Haut Koenigsbourg", "Colmar", "Eguisheim", "Besancon", "Dijon",
               "Annecy", "Grenoble", "Lyon", "Gorges du Verdon", "Bormes les Mimosas" ,"Cassis", "Marseille",
               "Aix en Provence", "Avignon", "Uzes", "Nimes", "Aigues Mortes", "Saintes Maries de la mer",
               "Collioure", "Carcassonne", "Ariege", "Toulouse", "Montauban", "Biarritz", "Bayonne", "La Rochelle"]

In [4]:
#Iteration on the studied list of cities 
#in order to get back their gps coordinates from the API https://nominatim.org/ 

list_gps = []

for element in list_cities:
    r_gps = requests.get("https://nominatim.openstreetmap.org/search?q={}&format=json".format(element)) #API request
    dic_gps = r_gps.json()
    lat = float(dic_gps[0]["lat"])
    lon = float(dic_gps[0]["lon"])
    list_gps.append([lat,lon]) #list containing latitude and longitude for each city

In [5]:
#The Dataframe will contain the gps coordinates as well as weather information for next 7 days for each city

columns = ["city_name", "city_id", "latitude", "longitude", "day_month", "weather_desc",
         "prob_precipitation", "daily_temperature"]

dataset = pd.DataFrame(columns = columns)
dataset

,city_name,city_id,latitude,longitude,day_month,weather_desc,prob_precipitation,daily_temperature


In [6]:
key_api = "0a436d07dcd54b9d7d4269ac00f935b8"

    
for j in range(len(list_cities)):
    
    city = list_cities[j]
    city_id = j
    lat = list_gps[j][0]
    lon = list_gps[j][1]
    
    #requesting on the API https://openweathermap.org/appid 
    r_weather = requests.get(
    "https://api.openweathermap.org/data/2.5/onecall?lat={}&lon={}&units={}&exclude=minutely,hourly&appid={}"\
        .format(lat, lon, 'metric', key_api)) 
    
    
    dic_weather = r_weather.json()
 
    dictionnary={'city_name': city,
                 'city_id':city_id,
                 'latitude':lat,
                 'longitude':lon}
    
    for i in range(1,8):
        timestamp = dic_weather["daily"][i]["dt"]
        date = datetime.fromtimestamp(timestamp)
        dictionnary['day_month'] = date.strftime("%d/%m")
        dictionnary['weather_desc'] = dic_weather["daily"][i]["weather"][0]["description"]
        dictionnary['daily_temperature'] = dic_weather["daily"][i]['temp']['day']
        dictionnary['prob_precipitation'] = float(dic_weather["daily"][i]["pop"])
        dataset = dataset.append(dictionnary, ignore_index = True)

In [7]:
dataset.head(10)

,city_name,city_id,latitude,longitude,day_month,weather_desc,prob_precipitation,daily_temperature
0,Mont Saint Michel,0,48.635523,-1.510257,21/09,few clouds,0.00,19.96
1,Mont Saint Michel,0,48.635523,-1.510257,22/09,few clouds,0.00,19.24
2,Mont Saint Michel,0,48.635523,-1.510257,23/09,broken clouds,0.00,18.50
3,Mont Saint Michel,0,48.635523,-1.510257,24/09,clear sky,0.00,18.28
4,Mont Saint Michel,0,48.635523,-1.510257,25/09,light rain,0.54,20.29
5,Mont Saint Michel,0,48.635523,-1.510257,26/09,light rain,0.93,19.44
6,Mont Saint Michel,0,48.635523,-1.510257,27/09,broken clouds,0.04,16.77
7,St Malo,1,48.649518,-2.026041,21/09,scattered clouds,0.00,17.38
8,St Malo,1,48.649518,-2.026041,22/09,few clouds,0.00,16.92
9,St Malo,1,48.649518,-2.026041,23/09,scattered clouds,0.00,17.37


In [8]:
#Computing the average temperature of the next 7 days for the different cities

mean_temp = dataset.groupby("city_name")["daily_temperature"]\
                .mean()\
                .reset_index()

mean_temp.rename(columns = {"city_name": "city_name", 
                          "daily_temperature": "average_temperature"}, 
                 inplace = True)
mean_temp

,city_name,average_temperature
0,Aigues Mortes,23.641429
1,Aix en Provence,24.820000
2,Amiens,18.374286
3,Annecy,19.448571
4,Ariege,15.335714
5,Avignon,24.838571
6,Bayeux,18.428571
7,Bayonne,20.487143
8,Besancon,18.087143
9,Biarritz,19.745714


In [9]:

#Adding the average temperature in the main dataset

dataset = dataset.merge(mean_temp, on = "city_name", how = "inner")
dataset

,city_name,city_id,latitude,longitude,day_month,weather_desc,prob_precipitation,daily_temperature,average_temperature
0,Mont Saint Michel,0,48.635523,-1.510257,21/09,few clouds,0.00,19.96,18.925714
1,Mont Saint Michel,0,48.635523,-1.510257,22/09,few clouds,0.00,19.24,18.925714
2,Mont Saint Michel,0,48.635523,-1.510257,23/09,broken clouds,0.00,18.50,18.925714
3,Mont Saint Michel,0,48.635523,-1.510257,24/09,clear sky,0.00,18.28,18.925714
4,Mont Saint Michel,0,48.635523,-1.510257,25/09,light rain,0.54,20.29,18.925714
...,...,...,...,...,...,...,...,...,...
240,La Rochelle,34,46.159113,-1.152043,23/09,clear sky,0.00,20.54,19.598571
241,La Rochelle,34,46.159113,-1.152043,24/09,clear sky,0.00,19.76,19.598571
242,La Rochelle,34,46.159113,-1.152043,25/09,light rain,0.20,18.44,19.598571
243,La Rochelle,34,46.159113,-1.152043,26/09,light rain,0.68,20.41,19.598571


In [10]:
#Saving the weather information in a CSV file
dataset.to_csv('villes - Projection meteo.csv', index = False)